In [1]:
import json
import os

In [ ]:
#Sample example for x264, x265 and libvpx encoder
#The source video (such as CSGO.y4m) is saved in a file namely 'source-video'
#After encoding, the results are saved in 'encode-video'
os.system('ffmpeg -i ./source-video/CSGO.y4m -c:v libx264 -x264-params crf=15 ./encode-video/x264/CSGO_libvpx.mp4')
os.system('ffmpeg -i ./source-video/CSGO.y4m -c:v libx265 -x265-params crf=15  ./encode-video/x265/CSGO_libvpx.mp4')
os.system('ffmpeg -i ./source-video/CSGO.y4m -c:v libvpx-vp9 -crf 14 -profile 0 ./encode-video/libvpx/CSGO_libvpx.mp4')

In [32]:
#A original video has 50 seconds which require nearly 2 hours on encoding.
#This process aims to cut the original video to 10 seconds in order to decrease the encoding time
#Output zero means the command is executed successfully.
os.system('ffmpeg -ss 0:0:0 -t 0:0:10 -i DOTA2.y4m -acodec copy DOTA2_output.y4m')

0

In [2]:
#This is the encoding process
#This section executes the VQ model
#The paramters of VQ model for three encoders are set. 
def encoder(source_video_file,codec_model,qp):
    #Get file name and format name
    filename,fileext = os.path.splitext(source_video_file)
    #example./ffmpeg -i ./source-video/bus_cif.y4m -c:v libx264 ./encode-video/libx264/bus_cif_libx264.mp4
    if codec_model=='libx264':
        command = 'ffmpeg -i ./source-video/' + source_video_file + ' -c:v libx264 ' + '-x264-params crf='+qp+':profile=High:preset=medium:keyint=4:min-keyint=4:me=2:ref=4:frame-threads=1:subme=5:aq-mode=1:aq-strength=1.0:psy-rd=1.0'+' ./encode-video/'+ codec_model + '/' + filename + '_' +codec_model+ '_qp'+qp+'.mp4'
    if codec_model=='libx265':
        command = 'ffmpeg -i ./source-video/' + source_video_file + ' -c:v libx265 ' + '-x265-params crf='+qp+':profile=main:preset=medium:keyint=4:min-keyint=4:me=2:ref=4:subme=5:pools=none:aq-mode=2:aq-strength=1.0:psy-rd=2.0:psy-rdoq=1.0'+ ' ./encode-video/'+ codec_model + '/' + filename + '_' +codec_model+'_qp'+qp+ '.mp4'
    if codec_model=='libvpx':
        #command = 'ffmpeg -i ./source-video/' + source_video_file + ' -c:v libvpx-vp9 '+ '-crf 14 -profile 0 -deadline best -end-usage vbr -kf-max-dist 4s -kf-min-dist 4s -lag-in-frames 16 -auto-alt-ref 1 -arnr-maxframes 7 -arnr-strength 5 -arnr-type 3 -static-thresh 0 -frame-parallel 0 -threads 1 -tile-columns 0 -tune psnr -aq-mode 0' + ' ./encode-video/'+ codec_model + '/' + filename + '_' +codec_model+ '.mp4'
        command = 'ffmpeg -i ./source-video/' + source_video_file + ' -c:v libvpx-vp9 '+ '-crf '+qp+' -profile 0 -deadline realtime -lag-in-frames 16 -auto-alt-ref 1 -arnr-maxframes 7 -arnr-strength 5 -arnr-type 3 -static-thresh 0 -frame-parallel 0 -threads 1 -tile-columns 0 -aq-mode 1' + ' ./encode-video/'+ codec_model + '/' + filename + '_' +codec_model+ '_qp'+qp+'.mp4'
    #print(command)
    os.system(command)

In [50]:
#Encode the uncompressed video source through x264,x265 and libvpx
source_video = os.listdir('./source-video')
# Set the source file, encoder type and QP values
#QP(x264)=QP(x265)={15,19,23,27,3,35,39,43}
#QP(libvpx-VP9)={14,21,28,35,42,49,56,63}
#Using another for loop to input the QP values can simplify the manual process.
#However, it can take more than 15 hours for each execution.
#Therefore, it is recommand to input the QP values manually. 
for file in source_video:
    encoder(file,'libx264','43')
    encoder(file,'libx265','43')
    encoder(file,'libvpx','63')

In [4]:
# Get the json file of VMAF evaluation standard and bit-rate
def getVmaf(source_video_file, codec_model,qp):
    filename,fileext = os.path.splitext(source_video_file)
    #Get vmaf evaluation standard
    #./ffmpeg -i ./encode-video/libx264/bus_cif_libx264.mp4 -i ./source-video/bus_cif.y4m -lavfi libvmaf="model_path=./model/vmaf_v0.6.1.pkl: psnr=1:ms_ssim=1:ssim=1:log_fmt=json:log_path= ./vmaf/bus_cif_libx264_vmaf.json" -f null -
    command_vmaf = 'ffmpeg -i ./encode-video/'+ codec_model +'/'+ filename + '_' + codec_model +'_qp'+qp+'.mp4' +' -i ./source-video/' + source_video_file + ' -lavfi libvmaf="model_path=./model/vmaf_v0.6.1.pkl: psnr=1:ms_ssim=1:ssim=1:log_fmt=json:log_path= ./vmaf/' + filename + '_' + codec_model + '_qp'+qp+'_vmaf.json" -f null -'
    
    #Get the format of the video to further get information such as bit-rate
    #./ffprobe -v quiet -print_format json -show_format ./encode-video/libx264/bus_cif_libx264.mp4 >> ./video-format/bus_cif_libx264_format.json
    command_format = 'ffprobe -v quiet -print_format json -show_format ./encode-video/' + codec_model + '/' + filename + '_' + codec_model +'_qp'+qp+ '.mp4 >> ./video-format/'+ filename + '_' + codec_model +'_qp'+qp+'_format.json'
    
    #print(command_vmaf)
    #print(command_format)
    os.system(command_vmaf)
    os.system(command_format)
    

In [51]:
# Set the source file, encoder type and QP values
#QP(x264)=QP(x265)={15,19,23,27,3,35,39,43}
#QP(libvpx-VP9)={14,21,28,35,42,49,56,63}
for file in source_video:
    getVmaf(file,'libx264','43')
    getVmaf(file,'libx265','43')
    getVmaf(file,'libvpx','63')

In [ ]:
#The following steps aim to get the information of VMAF package
#Include VMAF, PSNR,SSIM,MS-SSIM and bitrate.
#Use Bjøntegaard-Delta Rate method to evaluate these values in next part. 

In [7]:
#x264
with open("./vmaf/CSGO/CSGO_output_libx264_qp15_vmaf.json", 'r') as f:
    temp = json.loads(f.read())
    print(temp['VMAF score'])
    print(temp['PSNR score'])
    print(temp['SSIM score'])
    print(temp['MS-SSIM score']) 
with open("./video-format/CSGO/CSGO_output_libx264_qp15_format.json", 'r') as f:
    test = json.loads(f.read())
    print(test['format']['bit_rate'])

97.4751253415694
28019940


In [8]:
#x265
with open("./vmaf/CSGO/CSGO_output_libx265_qp15_vmaf.json", 'r') as f:
    temp = json.loads(f.read())
    print(temp['VMAF score'])
    print(temp['PSNR score'])
    print(temp['SSIM score'])
    print(temp['MS-SSIM score']) 
with open("./video-format/CSGO/CSGO_output_libx265_qp15_format.json", 'r') as f:
    test = json.loads(f.read())
    print(test['format']['bit_rate'])

95.55946840553997
17007813


In [43]:
#VP9
with open("./vmaf/CSGO/CSGO_output_libvpx_qp14_vmaf.json", 'r') as f:
    temp = json.loads(f.read())
    print(temp['VMAF score'])
    print(temp['PSNR score'])
    print(temp['SSIM score'])
    print(temp['MS-SSIM score']) 

with open("./video-format/CSGO/CSGO_output_libvpx_qp14_format.json", 'r') as f:
    test = json.loads(f.read())
    print(test['format']['bit_rate'])

91.97011480079937
4932790
